In [ ]:
import sys
import numpy as np
from numpy import loadtxt
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve
from sklearn.model_selection import train_test_split
import pandas as pd 
debug = True

f1_scores = []
fpr = []
tpr = []

print('Synthetic Dataset')

dataset = pd.read_csv("/kaggle/input/combined-dataset-before-smote/output-2.csv", header = None, delimiter = ',')
print(dataset)

X = dataset.drop(columns = [26,27,28,29])
y = dataset.filter([26,27,28,29])
print(X,y)

unique_rows = np.unique(y, axis=0).tolist()
print(unique_rows)

y['combined']= y.values.tolist()
y = y.drop(columns = [26,27,28,29])
print(y)
print(y.shape)
#merges mutliple targets into single multiclassfication for y
i = 0
while i < len(y):
    j = 0
    while j < len(unique_rows):
        if y.loc[y.index[i],'combined'] == unique_rows[j]:
            y.loc[y.index[i],'combined'] = unique_rows.index(unique_rows[j])
        j +=1
    i+=1
    
y['combined'] = y['combined'].astype(int)

y.to_numpy()

print(y)


# Split dataset into training and validation sets
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("done")
print(x_train.shape)
print(y_train.shape)



In [ ]:
import sys
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC

# Load the dataset
dataset = pd.read_csv("/kaggle/input/combined-dataset-before-smote/output-2.csv", header=None, delimiter=',')

# Assuming you've already split X and y
X = dataset.drop(columns=[26, 27, 28, 29])
y = dataset.filter([26, 27, 28, 29])

# Convert y to 1-dimensional array
y_labels = np.argmax(y.values, axis=1)


# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Parameter Tuning
#param_dist = {'C': np.logspace(-3, 3, 7)}  # Define a range of C values to search over

# Perform Randomized Search for hyperparameter tuning
#svm = SVC(kernel='linear')
#random_search = RandomizedSearchCV(svm, param_distributions=param_dist, n_iter=10, cv=5, n_jobs=-1)
#random_search.fit(X_scaled, y_labels)  # Use y_labels instead of y_subset

# Best C value
#best_C = random_search.best_params_['C']
#print("Best C value:", best_C)

print("Synthetic Model Start")

# Train the model with the best C value on the full dataset
svm_final = SVC(kernel='linear', C=1.0, verbose=True)
svm_final.fit(X_scaled, y_labels)

print("Synthetic Model Finished")

# Optionally, you can split the data into training and testing sets after scaling
x_train, x_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.metrics import roc_curve
from sklearn import metrics
print(x_test.shape)
y_pred = svm_final.predict(x_test)
#y_pred = to_categorical(y_pred)
y_test_labels = np.argmax(y_test.values, axis=1)

f1 = f1_score(y_test_labels, y_pred, average='weighted')
# Calculate accuracy and F1 score

accuracy = accuracy_score(y_test_labels, y_pred)
precision = precision_score(y_test_labels, y_pred, average='weighted')
recall = recall_score(y_test_labels, y_pred, average='weighted')

print("Accuracy:", accuracy)
print("F1 Score:", f1)
print("Precision Score:", precision)
print("Recall Score:", recall)

print (y_pred)
print (y_test)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.metrics import roc_curve
from sklearn import metrics

y_pred = svm_final.predict(x_test)
#y_pred = to_categorical(y_pred)
y_test_labels = np.argmax(y_test.values, axis=1)
print(y_pred)
y_pred_binary = np.argmax(y_pred)
f1 = f1_score(y_test_labels, y_pred, average='weighted')
# Calculate accuracy and F1 score

accuracy = accuracy_score(y_test_labels, y_pred)
precision = precision_score(y_test_labels, y_pred, average='weighted')
recall = recall_score(y_test_labels, y_pred, average='weighted')

cm = confusion_matrix(y_test_labels.ravel(), y_test_labels.ravel())
#np.savetxt('cm.txt', cm, delimiter=',', fmt='%f')
#f1 = f1_score(y_test.ravel(), y_pred.ravel())
tpr_value = cm[1, 1] / (cm[1, 1] + cm[1, 0])
fpr_value = cm[0, 1] / (cm[0, 1] + cm[0, 0])

print("Accuracy:", accuracy)
print("F1 Score:", f1)
print("Precision Score:", precision)
print("Recall Score:", recall)
print("TPR Score:", tpr_value)
print("FPR Score:", fpr_value)


In [ ]:
from sklearn.metrics import roc_curve, precision_recall_curve, auc
from sklearn.preprocessing import label_binarize
import numpy as np

# Assuming you have multi-class y_test_labels and y_pred
#y_test_labels = np.array()  # Example test labels
#y_pred = np.array()  # Example prediction scores (use predicted probabilities for each class)

# Binarize the labels
classes = np.unique(y_test_labels)
y_test_binarized = label_binarize(y_test_labels, classes=classes)

# For the ROC and PR curves, we'll use the binarized labels
# Compute PR and ROC curves for each class and calculate the AUC
prc_auc_dict = {}
roc_auc_dict = {}

for i, class_label in enumerate(classes):
    precision_a, recall_a, _ = precision_recall_curve(y_test_binarized[:, i], y_pred)
    prc_auc = auc(recall_a, precision_a)
    prc_auc_dict[class_label] = prc_auc

    fpr_a, tpr_a, _ = roc_curve(y_test_binarized[:, i], y_pred)
    roc_auc = auc(fpr_a, tpr_a)
    roc_auc_dict[class_label] = roc_auc

# Print AUC scores for each class
for class_label in classes:
    print(f"Class {class_label} - Area Under the PR Curve: {prc_auc_dict[class_label]:.4f}")
    print(f"Class {class_label} - Area Under the ROC Curve: {roc_auc_dict[class_label]:.4f}")


In [ ]:
import json

# Assuming tpr and fpr are NumPy arrays
print("Type of f1:", type(f1))
print("Type of precision:", type(precision))
print("Type of recall:", type(recall))
print("Type of tpr_value:", type(tpr_value))
print("Type of fpr_value:", type(fpr_value))
print("Type of roc_auc:", type(roc_auc))
print("Type of prc_auc:", type(prc_auc))
print("Type of tpr:", type(tpr))
print("Type of fpr:", type(fpr))

precision_list = precision_a.tolist()
recall_list = recall_a.tolist()
tpr_list = tpr_a.tolist()
fpr_list = fpr_a.tolist()

#print(f1.type())

data = {
    "Average F1 Score": float(f1), 
    "Average Precision": precision, 
    "Average Recall": float(recall), 
    "Average True Positive Rate": float(tpr_value), 
    "Average False Positive Rate": float(fpr_value), 
    "Average ROC AUC": roc_auc, 
    "Average PRC AUC": prc_auc,
    "TPR Array" : tpr_list,
    "FPR Array" : fpr_list,
    "Precision Array": precision_list, 
    "Recall Array": recall_list
}

# Specify the file path
file_path = "/kaggle/working/SVM - Synthetic - No SKF.json"

# Write the data to a JSON file
with open(file_path, "w") as json_file:
    json.dump(data, json_file)